## Read Transformed Data Files

In [0]:
spark.conf.set("###",dbutils.secrets.get(scope = "###", key = "###"))

In [0]:
filenames=dbutils.fs.ls("Retraining/TransformedData/")
filelist=[file.name for file in filenames if 'TransformedData_' in file.name]
filedates=[name.split('_')[1] for name in filelist]
filedates=[name.split('.')[0] for name in filedates]

In [0]:
import pandas as pd
pdfiledates = pd.Series(filedates)
pdfiledates = pd.to_datetime(pdfiledates, infer_datetime_format=True)

In [0]:
maxdate=str(pdfiledates.max()).replace(' ','T').replace(':','H',1).replace(':','m',1)+'s'

In [0]:
dffeatureEngTrain=spark.read.parquet("Retraining/TransformedData/TransformedData_"+maxdate+".parquet")

## Average for CountOf

In [0]:
import databricks.koalas as ks 
import numpy as np
dfAnalysisKTrain = ks.DataFrame(dffeatureEngTrain)

In [0]:
colnames=['CountOfInstrumentConditionBankruptcy',
 'CountOfInstrumentConditionInactive',
 'CountOfInstrumentConditionClosedByInstitution',
 'CountOfInstrumentStatusClosed',
 'CountOfInstrumentStatusOpen',
 'CountOfPaymentStatusChargedOff',
 'CountOfPaymentStatusNegativeShareBalance',
 'CountOfPaymentStatusPaidOff']

In [0]:
colnames2=['CountOfInstrumentConditionBankruptcy',
 'CountOfInstrumentConditionInactive',
 'CountOfInstrumentConditionClosedByInstitution',
 'CountOfInstrumentStatusClosed',
 'CountOfInstrumentStatusOpen',
 'CountOfPaymentStatusChargedOff',
 'CountOfPaymentStatusChargedOffPaidinFull',
 'CountOfPaymentStatusNegativeShareBalance',
 'CountOfPaymentStatusPaidOff']

In [0]:
POE_cols = []
for i in colnames:
  POE_cols= POE_cols+[col for col in dfAnalysisKTrain.columns if i in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

#### Drop NA

In [0]:
dfAnalysisFull2Train = dfAnalysisFullTrain.dropna()

#### Compute Average

In [0]:
for colname in colnames2:
  dfAnalysisFull2Train['1_to_4_avg_'+colname]=np.zeros(len(dfAnalysisFull2Train))

In [0]:
for colname in colnames2:
  for i in range(1,5):
    dfAnalysisFull2Train['1_to_4_avg_'+colname]=dfAnalysisFull2Train['1_to_4_avg_'+colname]+dfAnalysisFull2Train[str(i)+'_month_ago_'+colname]
  dfAnalysisFull2Train['1_to_4_avg_'+colname]=dfAnalysisFull2Train['1_to_4_avg_'+colname]/4

In [0]:
POE_cols= [col for col in dfAnalysisFull2Train.columns if 'avg' in col]
POE_cols.append('MemberIdentifier')

In [0]:
dfAnalysisFull3Train=pd.DataFrame()
dfAnalysisFull3Train=dfAnalysisFull2Train[POE_cols]

#### Write to file

In [0]:
dfpctchange2Train = ks.from_pandas(dfAnalysisFull3Train)
dfpctchange3Train = dfpctchange2Train.to_spark()

In [0]:
dfpctchange3Train = dfpctchange3Train.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3Train, dffeatureEngTrain.MemberIdentifier == dfpctchange3Train.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

##Average for Noof

In [0]:
colnames=['NoOfTransactionsBranch',
 'NoOfTransactionsOperations',
 'NoOfTransactionsContactCenter',
 'NoOfTransactionsIndirectLending',
 'NoOfTransactionsSharedBranch',
 'NoOfTransactionsMobileOnlineBanking',
 'NoOfTransactionsSystemGenerated',
 'NoOfTransactionsElectronicClearing',
 'NoOfTransactionsMerchantSign',
 'NoOfTransactionsLockbox']

In [0]:
POE_cols = []
for i in colnames:
  POE_cols= POE_cols+[col for col in dfAnalysisKTrain.columns if i in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
dfAnalysisFull2Train = dfAnalysisFullTrain.dropna()

#### Compute Average

In [0]:
for colname in colnames:
  dfAnalysisFull2Train['1_to_4_avg_'+colname]=np.zeros(len(dfAnalysisFull2Train))

In [0]:
for colname in colnames:
  for i in range(1,5):
    dfAnalysisFull2Train['1_to_4_avg_'+colname]=dfAnalysisFull2Train['1_to_4_avg_'+colname]+dfAnalysisFull2Train[str(i)+'_month_ago_'+colname]
  dfAnalysisFull2Train['1_to_4_avg_'+colname]=dfAnalysisFull2Train['1_to_4_avg_'+colname]/4

In [0]:
POE_cols= [col for col in dfAnalysisFull2Train.columns if 'avg' in col]
POE_cols.append('MemberIdentifier')

In [0]:
dfAnalysisFull3Train=pd.DataFrame()
dfAnalysisFull3Train=dfAnalysisFull2Train[POE_cols]

#### Write to file

In [0]:
dfpctchange2Train = ks.from_pandas(dfAnalysisFull3Train)
dfpctchange3Train = dfpctchange2Train.to_spark()

In [0]:
dfpctchange3Train = dfpctchange3Train.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3Train, dffeatureEngTrain.MemberIdentifier == dfpctchange3Train.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

##Days Since Engaged

In [0]:
colname = "DaysSinceEngaged"

In [0]:
POE_cols = [col for col in dfAnalysisKTrain.columns if colname in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
def tenure_lab(dfAnalysisFullTrain):
    if dfAnalysisFullTrain["0_month_ago_DaysSinceEngaged"] >= 1825:
        return 1825
    else:
        return dfAnalysisFullTrain["0_month_ago_DaysSinceEngaged"]
dfAnalysisFullTrain["DaysSinceEngaged"] = dfAnalysisFullTrain.apply(lambda dfAnalysisFullTrain:tenure_lab(dfAnalysisFullTrain),axis = 1)

In [0]:
dfAnalysisFullTrain = dfAnalysisFullTrain[['MemberIdentifier','DaysSinceEngaged']]

In [0]:
dfpctchange2 = ks.from_pandas(dfAnalysisFullTrain)
dfpctchange3 = dfpctchange2.to_spark()

In [0]:
dfpctchange3 = dfpctchange3.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3, dffeatureEngTrain.MemberIdentifier == dfpctchange3.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

##DebitCountLoan, CreditCountLoan

In [0]:
colnames = ['DebitCountLoan','CreditCountLoan']
colname = 'Net_CountLoan'

In [0]:
POE_cols0 = [col for col in dfAnalysisKTrain.columns if colnames[0] in col]
POE_cols1 = [col for col in dfAnalysisKTrain.columns if colnames[1] in col]
POE_cols = POE_cols0 + POE_cols1
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain.apply(pd.to_numeric)

#### Compute Net

In [0]:
for i in range(5):
  dfAnalysisFullTrain[str(i)+'_month_ago_'+colname]=dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[0]]+dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[1]]

#### Drop NA

In [0]:
dfdropna3Train = dfAnalysisFullTrain.dropna()

#### Compute Average

In [0]:
dfAnalysisFull2Train =dfdropna3Train.copy()

In [0]:
colnames.append('Net_CountLoan')

In [0]:
for colname in colnames:
  dfAnalysisFull2Train['1_to_4_avg_'+colname]=np.zeros(len(dfAnalysisFull2Train))

In [0]:
for colname in colnames:
  for i in range(1,5):
    dfAnalysisFull2Train['1_to_4_avg_'+colname]=dfAnalysisFull2Train['1_to_4_avg_'+colname]+dfAnalysisFull2Train[str(i)+'_month_ago_'+colname]
  dfAnalysisFull2Train['1_to_4_avg_'+colname]=dfAnalysisFull2Train['1_to_4_avg_'+colname]/4

In [0]:
POE_cols = [col for col in dfAnalysisFull2Train.columns if 'avg' in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("0_month_ago_Net_CountLoan")
dfpctchangeTrain = dfAnalysisFull2Train[POE_cols]

In [0]:
dfpctchange2Train = ks.from_pandas(dfpctchangeTrain)
dfpctchange3Train = dfpctchange2Train.to_spark()

#### Join Columns

In [0]:
dfpctchange3Train = dfpctchange3Train.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3Train, dffeatureEngTrain.MemberIdentifier == dfpctchange3Train.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## DebitCountShare, CreditCountShare

In [0]:
colnames = ['DebitCountShare','CreditCountShare']
colname = 'Net_CountShare'

In [0]:
POE_cols0 = [col for col in dfAnalysisKTrain.columns if colnames[0] in col]
POE_cols1 = [col for col in dfAnalysisKTrain.columns if colnames[1] in col]
POE_cols = POE_cols0 + POE_cols1
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain.apply(pd.to_numeric)

#### Compute Net

In [0]:
for i in range(5):
  dfAnalysisFullTrain[str(i)+'_month_ago_'+colname]=dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[0]]+dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[1]]

#### Drop NA

In [0]:
dfdropna3Train = dfAnalysisFullTrain.dropna()

#### Compute Average

In [0]:
dfAnalysisFull2Train =dfdropna3Train.copy()

In [0]:
colnames.append('Net_CountShare')

In [0]:
for colname in colnames:
  dfAnalysisFull2Train['1_to_4_avg_'+colname]=np.zeros(len(dfAnalysisFull2Train))

In [0]:
for colname in colnames:
  for i in range(1,5):
    dfAnalysisFull2Train['1_to_4_avg_'+colname]=dfAnalysisFull2Train['1_to_4_avg_'+colname]+dfAnalysisFull2Train[str(i)+'_month_ago_'+colname]
  dfAnalysisFull2Train['1_to_4_avg_'+colname]=dfAnalysisFull2Train['1_to_4_avg_'+colname]/4

In [0]:
POE_cols = [col for col in dfAnalysisFull2Train.columns if 'avg' in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("0_month_ago_Net_CountShare")
dfpctchangeTrain = dfAnalysisFull2Train[POE_cols]

In [0]:
dfpctchange2Train = ks.from_pandas(dfpctchangeTrain)
dfpctchange3Train = dfpctchange2Train.to_spark()

#### Join Columns

In [0]:
dfpctchange3Train = dfpctchange3Train.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3Train, dffeatureEngTrain.MemberIdentifier == dfpctchange3Train.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## HouseholdIdentifier

In [0]:
POE_cols = [col for col in dfAnalysisKTrain.columns if 'HouseholdIdentifier' in col]
POE_cols.append("MemberIdentifier")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
dfAnalysisFullTrain2 = dfAnalysisFullTrain[['0_month_ago_HouseholdIdentifier','1_month_ago_HouseholdIdentifier', '2_month_ago_HouseholdIdentifier','MemberIdentifier']]

In [0]:
for column in dfAnalysisFullTrain2.columns:
  x = dfAnalysisFullTrain2[column].value_counts().to_frame()
  x[column + '_2'] = x.index
  x = x.rename(columns = {column: column +'_count'})
  dfAnalysisFullTrain2 = pd.merge(dfAnalysisFullTrain2 , x, how ="left",left_on =column, right_on = column + '_2')

In [0]:
dfAnalysisFullTrain3 = dfAnalysisFullTrain2[['MemberIdentifier','0_month_ago_HouseholdIdentifier_count','1_month_ago_HouseholdIdentifier_count','2_month_ago_HouseholdIdentifier_count']]

In [0]:
colnames = ['1_month_ago_HouseholdIdentifier_count','2_month_ago_HouseholdIdentifier_count']

In [0]:
dfAnalysisFullTrain3['1_to_2_avg_HouseholdIdentifier_count']=np.zeros(len(dfAnalysisFullTrain3))

In [0]:
for i in range(1,3):
  dfAnalysisFullTrain3['1_to_2_avg_HouseholdIdentifier_count'] = dfAnalysisFullTrain3['1_to_2_avg_HouseholdIdentifier_count'] + dfAnalysisFullTrain3[str(i)+'_month_ago_HouseholdIdentifier_count']

In [0]:
dfAnalysisFullTrain3['1_to_2_avg_HouseholdIdentifier_count']=dfAnalysisFullTrain3['1_to_2_avg_HouseholdIdentifier_count']/2

In [0]:
dfAnalysisFullTrain3 = dfAnalysisFullTrain3[["MemberIdentifier", "0_month_ago_HouseholdIdentifier_count", "1_to_2_avg_HouseholdIdentifier_count"]]

In [0]:
dfpctchange2 = ks.from_pandas(dfAnalysisFullTrain3)
dfpctchange3 = dfpctchange2.to_spark()

In [0]:
dfpctchange3 = dfpctchange3.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3, dffeatureEngTrain.MemberIdentifier == dfpctchange3.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## Net for AmountLoan

In [0]:
colnames = ['DebitAmountLoan','CreditAmountLoan']
colname = 'Net_AmountLoan'

In [0]:
POE_cols0 = [col for col in dfAnalysisKTrain.columns if colnames[0] in col]
POE_cols1 = [col for col in dfAnalysisKTrain.columns if colnames[1] in col]
POE_cols = POE_cols0 + POE_cols1
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain.apply(pd.to_numeric)

#### Compute Net

In [0]:
for i in range(5):
  dfAnalysisFullTrain[str(i)+'_month_ago_'+colname]=dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[0]]+dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[1]]

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain[[col for col in dfAnalysisFullTrain.columns if colname in col]+['TARGET','MemberIdentifier']]

#### Drop NA

In [0]:
dfdropna3Train = dfAnalysisFullTrain.dropna()

#### Compute Average

In [0]:
dfAnalysisFull2Train =dfdropna3Train.copy()

In [0]:
for i in range(1,5,2):
  dfAnalysisFull2Train[str(i)+'_to_'+str(i+1)+'_avg_'+colname]=(dfAnalysisFull2Train[str(i)+'_month_ago_'+colname]+dfAnalysisFull2Train[str(i+1)+'_month_ago_'+colname])/2

In [0]:
POE_cols = [col for col in dfAnalysisFull2Train.columns if 'avg' in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("0_month_ago_Net_AmountLoan")
dfpctchangeTrain = dfAnalysisFull2Train[POE_cols]

In [0]:
dfpctchange2Train = ks.from_pandas(dfpctchangeTrain)
dfpctchange3Train = dfpctchange2Train.to_spark()

#### Join Columns

In [0]:
dfpctchange3Train = dfpctchange3Train.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3Train, dffeatureEngTrain.MemberIdentifier == dfpctchange3Train.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## Net for AmountLoan Updated

In [0]:
colnames = ['DebitAmountLoan','CreditAmountLoan']
colname = 'Net_AmountLoan'

In [0]:
POE_cols0 = [col for col in dfAnalysisKTrain.columns if colnames[0] in col]
POE_cols1 = [col for col in dfAnalysisKTrain.columns if colnames[1] in col]
POE_cols = POE_cols0 + POE_cols1
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain.apply(pd.to_numeric)

#### Compute Net

In [0]:
for i in range(5):
  dfAnalysisFullTrain[str(i)+'_month_ago_'+colname]=dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[0]]+dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[1]]

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain[[col for col in dfAnalysisFullTrain.columns if colname in col]+['TARGET','MemberIdentifier']]

#### Drop NA

In [0]:
dfdropna3Train = dfAnalysisFullTrain.dropna()

#### Compute Percentage Change

In [0]:
dfAnalysisFull2Train =dfdropna3Train.copy()

In [0]:
zero_cols=['0_month_ago_Net_AmountLoan', '1_month_ago_Net_AmountLoan','2_month_ago_Net_AmountLoan', '3_month_ago_Net_AmountLoan','4_month_ago_Net_AmountLoan']

In [0]:
for col in zero_cols:
  dfAnalysisFull2Train[col].replace(0,0.01, inplace=True)

In [0]:
for i in range(4):
  dfAnalysisFull2Train[str(i)+'_to_'+str(i+1)+'_pct_change_'+colname]=np.nan_to_num((dfAnalysisFull2Train[str(i)+'_month_ago_'+colname]-dfAnalysisFull2Train[str(i+1)+'_month_ago_'+colname])*100/(dfAnalysisFull2Train[str(i+1)+'_month_ago_'+colname]))

In [0]:
POE_cols = [col for col in dfAnalysisFull2Train.columns if 'pct_change' in col]
POE_cols.append("MemberIdentifier")
dfpctchangeTrain = dfAnalysisFull2Train[POE_cols]

In [0]:
dfpctchange2Train = ks.from_pandas(dfpctchangeTrain)
dfpctchange3Train = dfpctchange2Train.to_spark()

#### Join Columns

In [0]:
dfpctchange3Train = dfpctchange3Train.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3Train, dffeatureEngTrain.MemberIdentifier == dfpctchange3Train.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## Net for AmountShare

In [0]:
colnames = ['DebitAmountShare','CreditAmountShare']
colname = 'Net_AmountShare'

In [0]:
POE_cols0 = [col for col in dfAnalysisKTrain.columns if colnames[0] in col]
POE_cols1 = [col for col in dfAnalysisKTrain.columns if colnames[1] in col]
POE_cols = POE_cols0 + POE_cols1
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain.apply(pd.to_numeric)

#### Compute Net

In [0]:
for i in range(5):
  dfAnalysisFullTrain[str(i)+'_month_ago_'+colname]=dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[0]]+dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[1]]

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain[[col for col in dfAnalysisFullTrain.columns if colname in col]+['TARGET','MemberIdentifier']]

#### Drop NA

In [0]:
dfdropna3Train = dfAnalysisFullTrain.dropna()

#### Compute Average

In [0]:
dfAnalysisFull2Train =dfdropna3Train.copy()

In [0]:
for i in range(1,5,2):
  dfAnalysisFull2Train[str(i)+'_to_'+str(i+1)+'_avg_'+colname]=(dfAnalysisFull2Train[str(i)+'_month_ago_'+colname]+dfAnalysisFull2Train[str(i+1)+'_month_ago_'+colname])/2

In [0]:
POE_cols = [col for col in dfAnalysisFull2Train.columns if 'avg' in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("0_month_ago_Net_AmountShare")
dfpctchangeTrain = dfAnalysisFull2Train[POE_cols]

In [0]:
dfpctchange2Train = ks.from_pandas(dfpctchangeTrain)
dfpctchange3Train = dfpctchange2Train.to_spark()

#### Join Columns

In [0]:
dfpctchange3Train = dfpctchange3Train.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3Train, dffeatureEngTrain.MemberIdentifier == dfpctchange3Train.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## Net for AmountShare Updated

In [0]:
colnames = ['DebitAmountShare','CreditAmountShare']
colname = 'Net_AmountShare'

In [0]:
POE_cols0 = [col for col in dfAnalysisKTrain.columns if colnames[0] in col]
POE_cols1 = [col for col in dfAnalysisKTrain.columns if colnames[1] in col]
POE_cols = POE_cols0 + POE_cols1
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain.apply(pd.to_numeric)

#### Compute Net

In [0]:
for i in range(5):
  dfAnalysisFullTrain[str(i)+'_month_ago_'+colname]=dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[0]]+dfAnalysisFullTrain[str(i)+'_month_ago_'+colnames[1]]

In [0]:
dfAnalysisFullTrain=dfAnalysisFullTrain[[col for col in dfAnalysisFullTrain.columns if colname in col]+['TARGET','MemberIdentifier']]

#### Drop NA

In [0]:
dfdropna3Train = dfAnalysisFullTrain.dropna()

#### Compute Percentage Change

In [0]:
dfAnalysisFull2Train =dfdropna3Train.copy()

In [0]:
zero_cols=['0_month_ago_Net_AmountShare', '1_month_ago_Net_AmountShare','2_month_ago_Net_AmountShare', '3_month_ago_Net_AmountShare','4_month_ago_Net_AmountShare']

In [0]:
for col in zero_cols:
  dfAnalysisFull2Train[col].replace(0,0.01, inplace=True)

In [0]:
for i in range(4):
  dfAnalysisFull2Train[str(i)+'_to_'+str(i+1)+'_pct_change_'+colname]=np.nan_to_num((dfAnalysisFull2Train[str(i)+'_month_ago_'+colname]-dfAnalysisFull2Train[str(i+1)+'_month_ago_'+colname])*100/(dfAnalysisFull2Train[str(i+1)+'_month_ago_'+colname]))

In [0]:
POE_cols= [col for col in dfAnalysisFull2Train.columns if 'pct_change' in col]
POE_cols.append("MemberIdentifier")
dfpctchangeTrain = dfAnalysisFull2Train[POE_cols]

In [0]:
dfpctchange2Train = ks.from_pandas(dfpctchangeTrain)
dfpctchange3Train = dfpctchange2Train.to_spark()

#### Join Columns

In [0]:
dfpctchange3Train = dfpctchange3Train.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3Train, dffeatureEngTrain.MemberIdentifier == dfpctchange3Train.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## PctChangeUpdated

In [0]:
colname = "ProdAmountPOE"

In [0]:
POE_cols = [col for col in dfAnalysisKTrain.columns if colname in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

####Remove Missing Values

In [0]:
dfdropna3Train = dfAnalysisFullTrain.dropna()

#### Compute Percentage Change

In [0]:
dfAnalysisFull2Train =dfdropna3Train.copy()

In [0]:
dfAnalysisFull2Train=dfAnalysisFull2Train.apply(pd.to_numeric)

In [0]:
colnames=['ProdAmountPOETransactional',
'ProdAmountPOEWealth',
'ProdAmountPOERealEstate',
'ProdAmountPOESavings',
'ProdAmountPOELoans',
]
zero_cols=[]
for colname in colnames:
  zero_cols=zero_cols+[col for col in dfAnalysisFull2Train.columns if colname in col]

In [0]:
for col in zero_cols:
  dfAnalysisFull2Train[col].replace(0,0.01, inplace=True)

In [0]:
for colname in colnames:
  for i in range(4):
    dfAnalysisFull2Train[str(i)+'_to_'+str(i+1)+'_pct_change_'+colname]=np.nan_to_num((dfAnalysisFull2Train[str(i)+'_month_ago_'+colname]-dfAnalysisFull2Train[str(i+1)+'_month_ago_'+colname])*100/(dfAnalysisFull2Train[str(i+1)+'_month_ago_'+colname]))

In [0]:
POE_cols = [col for col in dfAnalysisFull2Train if 'pct_change' in col]
POE_cols.append("MemberIdentifier")
dfpctchangeTrain = dfAnalysisFull2Train[POE_cols]

In [0]:
dfpctchange2Train = ks.from_pandas(dfpctchangeTrain)
dfpctchange3Train = dfpctchange2Train.to_spark()

#### Join Columns

In [0]:
dfpctchange3Train = dfpctchange3Train.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3Train, dffeatureEngTrain.MemberIdentifier == dfpctchange3Train.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## Poe AVG Amounts (Added loop to cover all columns, verify)

In [0]:
colname = "ProdAmountPOE"

In [0]:
POE_cols = [col for col in dfAnalysisKTrain.columns if colname in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

#### Drop NA

In [0]:
dfdropnaTrain3 = dfAnalysisFullTrain.dropna()

In [0]:
dfdropnaTrain3=dfdropnaTrain3.apply(pd.to_numeric)

In [0]:
POE_cols = [col for col in dfdropnaTrain3.columns if 'month_ago' in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain2 = dfdropnaTrain3[POE_cols]

In [0]:
colnames=["ProdAmountPOERealEstate","ProdAmountPOELoans","ProdAmountPOETransactional","ProdAmountPOESavings","ProdAmountPOEWealth"]

In [0]:
for colname in colnames:  
  for i in range(1,5,2):
    dfAnalysisFullTrain2[str(i)+'_to_'+str(i+1)+'_avg_'+colname]=(dfAnalysisFullTrain2[str(i)+'_month_ago_'+colname]+dfAnalysisFullTrain2[str(i+1)+'_month_ago_'+colname])/2

In [0]:
POE_cols = [col for col in dfAnalysisFullTrain2.columns if 'avg' in col]
POE_cols.append("MemberIdentifier")
dfpctchangetrain = dfAnalysisFullTrain2[POE_cols]

In [0]:
dfpctchange2 = ks.from_pandas(dfpctchangetrain)
dfpctchange3 = dfpctchange2.to_spark()

#### Join Columns

In [0]:
dfpctchange3 = dfpctchange3.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3, dffeatureEngTrain.MemberIdentifier == dfpctchange3.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## POE Counts (Added loop, verify)

In [0]:
colname = "ProdCountPOE"

In [0]:
POE_cols = [col for col in dfAnalysisKTrain.columns if colname in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

#### Drop NA

In [0]:
dfdropnaTrain3 = dfAnalysisFullTrain.dropna()

In [0]:
dfdropnaTrain3=dfdropnaTrain3.apply(pd.to_numeric)

In [0]:
dfpctchangeTrain = dfdropnaTrain3.copy()

In [0]:
colnames = ["ProdCountPOEWealth","ProdCountPOESavings","ProdCountPOERealEstate","ProdCountPOELoans","ProdCountPOETransactional"]

In [0]:
for colname in colnames:
  dfpctchangeTrain['1_to_4_avg_'+colname]=np.zeros(len(dfpctchangeTrain))

In [0]:
for colname in colnames:
  for i in range(1,5):
    dfpctchangeTrain['1_to_4_avg_'+colname]=dfpctchangeTrain['1_to_4_avg_'+colname]+dfpctchangeTrain[str(i)+'_month_ago_'+colname]

In [0]:
for colname in colnames:
  dfpctchangeTrain['1_to_4_avg_'+colname]= dfpctchangeTrain['1_to_4_avg_'+colname]/4

In [0]:
POE_cols = [col for col in  dfpctchangeTrain.columns if 'avg' in col]
POE_cols.append("MemberIdentifier")
dfpctchangeTrain =  dfpctchangeTrain[POE_cols]

In [0]:
dfpctchange2 = ks.from_pandas(dfpctchangeTrain)
dfpctchange3 = dfpctchange2.to_spark()

#### Join Column

In [0]:
dfpctchange3 = dfpctchange3.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3, dffeatureEngTrain.MemberIdentifier == dfpctchange3.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## POE votingclassifying

In [0]:
POE_cols = POE_cols0 = [col for col in dfAnalysisKTrain.columns if "month_ago_POE" in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
colnames=["POERealEstate","POELoans","POETransactional","POESavings","POEWealth"]

In [0]:
cols=[]
for colname in colnames:
  l=[]
  for i in range(1,5):
    l.append(str(i)+"_month_ago_"+colname)
  cols.append(l)

In [0]:
cols2=[]
for colname in colnames:
  for i in range(5):
    cols2.append(str(i)+"_month_ago_"+colname)

In [0]:
dfAnalysis5 = dfAnalysisFullTrain[cols2]

In [0]:
for colname in cols:
  dfAnalysis5['voting_1_4'+colname[0].split('_')[-1]]=dfAnalysis5[colname].max(axis=1)

In [0]:
dfAnalysis5['MemberIdentifier']=dfAnalysisFullTrain['MemberIdentifier']

In [0]:
POE_cols = [col for col in  dfAnalysis5.columns if 'voting_1_4' in col]
POE_cols.append("MemberIdentifier")
dfpctchangetrain = dfAnalysis5[POE_cols]

In [0]:
dfpctchange2 = ks.from_pandas(dfpctchangetrain)
dfpctchange3 = dfpctchange2.to_spark()

In [0]:
dfpctchange3 = dfpctchange3.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3, dffeatureEngTrain.MemberIdentifier == dfpctchange3.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## Voting HasColumns

In [0]:
POE_cols = [col for col in dfAnalysisKTrain.columns if "month_ago_Has" in col]
POE_cols.append("MemberIdentifier")
POE_cols.append("TARGET")
dfAnalysisFullTrain = dfAnalysisKTrain[POE_cols].to_pandas()

In [0]:
colnames=["HasNoMarketingMail","HasFPSInvestment","HasDirectDeposit","HasOnlineBanking","HasNoMarketing","HasCheckingRewards"]

In [0]:
cols=[]
for colname in colnames:
  l=[]
  for i in range(1,5):
    l.append(str(i)+"_month_ago_"+colname)
  cols.append(l)

In [0]:
cols2=[]
for colname in colnames:
  for i in range(5):
    cols2.append(str(i)+"_month_ago_"+colname)

In [0]:
dfAnalysis5 = dfAnalysisKTrain[cols2].to_pandas()

In [0]:
for colname in cols:
  dfAnalysis5['voting_1_4'+colname[0].split('_')[-1]]=dfAnalysis5[colname].max(axis=1)

In [0]:
dfAnalysis5['MemberIdentifier']=dfAnalysisFullTrain['MemberIdentifier']

In [0]:
POE_cols = [col for col in  dfAnalysis5.columns if 'voting_1_4' in col]
POE_cols.append("MemberIdentifier")
dfpctchangetrain = dfAnalysis5[POE_cols]

In [0]:
dfpctchange2 = ks.from_pandas(dfpctchangetrain)
dfpctchange3 = dfpctchange2.to_spark()

In [0]:
dfpctchange3 = dfpctchange3.withColumnRenamed('MemberIdentifier','MemberIdentifier_2')

In [0]:
dffeatureEngTrain = dffeatureEngTrain.join(dfpctchange3, dffeatureEngTrain.MemberIdentifier == dfpctchange3.MemberIdentifier_2, how='left').drop('MemberIdentifier_2')

## Final write of data

In [0]:
# dffeatureEngTrainks=ks.DataFrame(dffeatureEngTrain)
dffeatureEngTrainpd=dffeatureEngTrain.toPandas()
# dffeatureEngTrainks=dffeatureEngTrainks.apply(ks.to_numeric)
dffeatureEngTrainpd=dffeatureEngTrainpd.apply(pd.to_numeric)
dffeatureEngTrainks=ks.from_pandas(dffeatureEngTrainpd)
dffeatureEngTrain=dffeatureEngTrainks.to_spark()

In [0]:
from pyspark.sql.types import IntegerType
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_POERealEstate", dffeatureEngTrain["0_month_ago_POERealEstate"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_POELoans", dffeatureEngTrain["0_month_ago_POELoans"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_POESavings", dffeatureEngTrain["0_month_ago_POESavings"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_POETransactional", dffeatureEngTrain["0_month_ago_POETransactional"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_POEWealth", dffeatureEngTrain["0_month_ago_POEWealth"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4POEWealth", dffeatureEngTrain["voting_1_4POEWealth"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4POELoans", dffeatureEngTrain["voting_1_4POELoans"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4POERealEstate", dffeatureEngTrain["voting_1_4POERealEstate"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4POESavings", dffeatureEngTrain["voting_1_4POESavings"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4POETransactional", dffeatureEngTrain["voting_1_4POETransactional"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4HasFPSInvestment", dffeatureEngTrain["voting_1_4HasFPSInvestment"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4HasDirectDeposit", dffeatureEngTrain["voting_1_4HasDirectDeposit"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4HasOnlineBanking", dffeatureEngTrain["voting_1_4HasOnlineBanking"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4HasNoMarketing", dffeatureEngTrain["voting_1_4HasNoMarketing"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("voting_1_4HasCheckingRewards", dffeatureEngTrain["voting_1_4HasCheckingRewards"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_HasNoMarketing", dffeatureEngTrain["0_month_ago_HasNoMarketing"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_HasFPSInvestment", dffeatureEngTrain["0_month_ago_HasFPSInvestment"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_HasCheckingRewards", dffeatureEngTrain["0_month_ago_HasCheckingRewards"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_HasDirectDeposit", dffeatureEngTrain["0_month_ago_HasDirectDeposit"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_HasOnlineBanking", dffeatureEngTrain["0_month_ago_HasOnlineBanking"].cast(IntegerType()))
dffeatureEngTrain = dffeatureEngTrain.withColumn("0_month_ago_HasNoMarketingMail", dffeatureEngTrain["0_month_ago_HasNoMarketingMail"].cast(IntegerType()))

In [0]:
dffeatureEngTrain.write.mode("overwrite").format("com.databricks.spark.parquet").parquet("Retraining/FeatureEngineeringData/FeatureEngineeringData_"+maxdate+".parquet")

## Creating and Writing Means Data

In [0]:
dfAnalysisKTrainFinal = ks.DataFrame(dffeatureEngTrain)

In [0]:
# meandfks = dfAnalysisKTrainFinal.mean().to_frame().transpose()
meandfks=dfAnalysisKTrainFinal.describe()
meandfks['ROC']=0.0
meandfks['RunID']=''
meandfks['TimeOfCreation']=maxdate
meandfks.index.name="Index"
meandfks.reset_index(inplace=True)

In [0]:
def path_exists(path):
  try:
    if len(dbutils.fs.ls(path)) > 0:
      return True
  except:
    return False

In [0]:
path="Retraining/DashboardData/DashboardData.parquet"
if path_exists(path):
  dfMeans=spark.read.parquet(path)
  dfMeansks=ks.DataFrame(dfMeans)
  dfMeansks=ks.concat([dfMeansks, meandfks])
  dfMeansks = dfMeansks.to_spark()
else:
  dfMeansks = meandfks.to_spark()

In [0]:
dfMeansks.write.mode("overwrite").format("com.databricks.spark.parquet").parquet("Retraining/DashboardData/DashboardData.parquet")